In [5]:
# using a conda environment Python=3.9
# running on Intel MacOS using Apple Metal for TF optimization
# using TensorFlow as ML framework
# versions:
from tensorflow import __version__ as tf_version
print("Python: ", sys.version)
print("TensorFlow: ", tf_version)

Python:  3.9.13 (main, Oct 13 2022, 16:12:30) 
[Clang 12.0.0 ]
TensorFlow:  2.10.0


In [13]:
# load MNIST using TF Load
import tensorflow as tf
import tensorflow_datasets as tfds
mnist = tf.keras.datasets.mnist
(train_ds, val_ds, test_ds), ds_info = tfds.load(
    'mnist',
    split=['train[:80%]', 'train[80%:90%]', 'train[90%:]'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)
num_classes = ds_info.features['label'].num_classes
# 10 classes for [0...9]
num_classes

10